In [16]:
import warnings
warnings.filterwarnings("ignore")

import unyt as u

from flowermd.base import Pack, Simulation
from flowermd.library.polymers import PPS, LJChain, EllipsoidChain
from flowermd.library.forcefields import BeadSpring, KremerGrestBeadSpring, EllipsoidForcefield, OPLS_AA_PPS
from flowermd.utils.rigid_body import create_rigid_body

In [18]:
pps_chains = PPS(num_mols=25, lengths=20)

bead_spring_chains = LJChain(num_mols=25, lengths=20)

ellipsoid_chains = EllipsoidChain(num_mols=25, lengths=20, lpar=1, bead_mass=1.0, bond_length=0.1)

In [19]:
pps_system = Pack(molecules=pps_chains, density=1.2 * u.g/(u.cm**3))

bead_spring_system = Pack(molecules=bead_spring_chains, density=0.6/u.nm**3)

ellipsoid_system = Pack(molecules=ellipsoid_chains, density=0.6/u.nm**3)
rigid_frame, rigid = create_rigid_body(
    ellipsoid_system.hoomd_snapshot,
    ellipsoid_chains.bead_constituents_types,
    bead_name="R",
)

In [20]:
pps_system.apply_forcefield(force_field=OPLS_AA_PPS(), r_cut=2.5, auto_scale=True)

bead_spring_ff = BeadSpring(
    beads={"A": dict(epsilon=1.0, sigma=1.0)},
    bonds={"A-A": dict(k=300, r0=1.1)},
    angles={"A-A-A": dict(k=300, t0=2.6)},
    r_cut=2.5
)

ellipsoid_ff = EllipsoidForcefield(
    lpar=1.0,
    lperp=0.4,
    epsilon=1.0,
    r_cut=2.5,
    bond_k=300,
    bond_r0=0.1,
    angle_k=150,
    angle_theta0=2.6
)

In [21]:
pps_sim = Simulation(initial_state=pps_system.hoomd_snapshot, forcefield=pps_system.hoomd_forcefield)
bead_spring_sim = Simulation(initial_state=bead_spring_system.hoomd_snapshot, forcefield=bead_spring_ff.hoomd_forces)
ellipsoid_sim = Simulation(initial_state=rigid_frame, forcefield=ellipsoid_ff.hoomd_forces, rigid_constraint=rigid)

Initializing simulation state from a gsd.hoomd.Frame.
Initializing simulation state from a gsd.hoomd.Frame.
Initializing simulation state from a gsd.hoomd.Frame.


In [22]:
pps_sim.run_NVT(kT=1.2, tau_kt=100 * pps_sim.dt, n_steps=1)
bead_spring_sim.run_NVT(kT=1.2, tau_kt=100 * bead_spring_sim.dt, n_steps=1)
ellipsoid_sim.run_NVT(kT=1.2, tau_kt=100 * ellipsoid_sim.dt, n_steps=1)

*Warning*: charge.pppm: system is not neutral and unscreened interactions are calculated, the net charge is -4636.93
*Warning*: charge.pppm: RMS error of 1.12531 is probably too high! 1.12531 1.12529
